In [1]:
import numpy as np
import tensorflow as tf
import glob 
import pandas as pd
import h5py # import to read hdf5
from pathlib import Path
from tqdm import tqdm

2022-11-03 17:37:52.058770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 17:37:52.377225: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-03 17:37:52.377289: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-03 17:37:52.441782: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 17:37:53.797541: W tensorflow/stream_executor/platform/de

In [2]:
# # Load /Users/viktorcikojevic/Documents/gravitational-waves-detection-kaggle/train_labels.csv as pandas dataframe
# df = pd.read_csv('/input/test_labels.csv')
# root_folder = "/input/test"


# # Rename the column 'id' to 'filename'. Add "root_folder" to the beginning of the filename

df = pd.DataFrame({'filename': glob.glob('/input/test/*.hdf5')})
df.head(5)

,filename
0,/input/test/e63b7654c.hdf5
1,/input/test/5f92b77cb.hdf5
2,/input/test/7fa2c724f.hdf5
3,/input/test/0d9fd6c93.hdf5
4,/input/test/a12046e5a.hdf5


In [3]:
len(df)

7975

In [4]:
df.loc[0]['filename']

'/input/test/e63b7654c.hdf5'

# Read file function 👀

In [5]:

# Each vector x has shape (4, 360, N). If N is not a multiple of 360, then we need to pad the vector with random noise with mu=np.average(x) and sigma=np.std(x)
def pad_amplitudes(x1, x2):
    # Get the shape of the vectors
    shape1 = x1.shape
    shape2 = x2.shape
    # Get the number of elements in each vector
    n1 = shape1[1]
    n2 = shape2[1]
    # Get the number of elements to pad
    n_pad = abs(n1 - n2)
    # Get the average and std of each vector
    mu1 = np.average(x1)
    mu2 = np.average(x2)
    sigma1 = np.std(x1)
    sigma2 = np.std(x2)
    # Get the padding vector
    pad1 = np.random.normal(mu1, sigma1, (360, n_pad))
    pad2 = np.random.normal(mu2, sigma2, (360, n_pad))
    # Pad the shorter vector
    if n1 > n2:
        x2 = np.concatenate((x2, pad2), axis=1)
    else:
        x1 = np.concatenate((x1, pad1), axis=1)
    return x1, x2


def pad_vector(x):
    if x.shape[2] % 360 != 0:
        mu = np.average(x)
        sigma = np.std(x)
        padding = np.random.normal(mu, sigma, (4, x.shape[1],360 - x.shape[2] % 360))
        x = np.concatenate((x, padding), axis=1)
    return x

def combine_amplitudes(amplitude_0, amplitude_1):
    
    amplitude_0, amplitude_1 = pad_amplitudes(amplitude_0, amplitude_1)
    # Get the real part of the amplitudes
    real_0 = amplitude_0.real
    real_1 = amplitude_1.real
    # Get the imaginary part of the amplitudes
    imag_0 = amplitude_0.imag
    imag_1 = amplitude_1.imag
    # normalize the amplitudes to be between 0 and 1
    real_0 = (real_0 - real_0.min()) / (real_0.max() - real_0.min())
    real_1 = (real_1 - real_1.min()) / (real_1.max() - real_1.min())
    imag_0 = (imag_0 - imag_0.min()) / (imag_0.max() - imag_0.min())
    imag_1 = (imag_1 - imag_1.min()) / (imag_1.max() - imag_1.min())
    # Expand the dims of the real and imaginary parts
    real_0 = np.expand_dims(real_0, axis=0)
    real_1 = np.expand_dims(real_1, axis=0)
    imag_0 = np.expand_dims(imag_0, axis=0)
    imag_1 = np.expand_dims(imag_1, axis=0)
    # Make a vstack of the amplitudes
    amplitudes = np.vstack((real_0, imag_0, real_1, imag_1))
    return amplitudes



# Idea from this function takes from this notebook (😇): https://www.kaggle.com/code/ayuraj/g2net-understand-the-data
def read_data(file):
    file = Path(file)
    with h5py.File(file, "r") as f:
        filename = file.stem
        f = f[filename]
        h1 = f["H1"]
        l1 = f["L1"]
        freq_hz = list(f["frequency_Hz"])
        
        h1_stft = h1["SFTs"][()]
        h1_timestamp = h1["timestamps_GPS"][()]
        # H2 data
        l1_stft = l1["SFTs"][()]
        l1_timestamp = l1["timestamps_GPS"][()]
        
        return combine_amplitudes(h1_stft, l1_stft)


In [6]:
# x = read_data("/input/train/001121a05.hdf5")

In [7]:
# Each vector x has shape (4, 360, N). If N is not a multiple of 360, then we need to pad the vector with random noise with mu=np.average(x) and sigma=np.std(x)
def pad_vector(x):
    if x.shape[1] % 360 != 0:
        mu = np.average(x)
        sigma = np.std(x)
        padding = np.random.normal(mu, sigma, (4, 360 - x.shape[1] % 360, x.shape[2]))
        x = np.concatenate((x, padding), axis=1)
    return x

In [8]:
# Each vector x has shape (4, 360, N). Loop over the vector and slice it into vectors of shape (4, 360, 360). If N is not a multiple of 360, then we need to pad the vector with random noise with mu=np.average(x) and sigma=np.std(x) before slicing
# Return a list of vectors
def slice_vector(x):
    x = pad_vector(x)
    slices = []
    for i in range(x.shape[2] // 360):
        slices.append(x[:, :, i * 360 : (i + 1) * 360])
    return np.array(slices)

In [9]:
# For a given vector (M, 4, 360, 360) and model, return the predictions of the model

def get_predictions(x, model):
    x = tf.convert_to_tensor(x)
    return model.predict(x)
    predictions = []
    for i in range(x.shape[0]):
        predictions.append(model.predict(x[i].reshape(1, 4, 360, 360)))
    return np.array(predictions)

In [10]:

def get_model(path):
    # Create a ResNet model. Load from keras.applications.resnet50.ResNet50
    from tensorflow.keras.applications.resnet50 import ResNet50
    # Create a ResNet50 model with weights not pre-trained on  ImageNet. Set include_top to False.  Set input_shape to (360, 360, 4). Set pooling to 'avg'. Classify into 2 classes. Activate sigmoid function.
    resnet = ResNet50(weights=None, include_top=False, input_shape=(360, 360, 4), pooling='avg', classes=1)
    model = tf.keras.Sequential([
        tf.keras.layers.Permute((2, 3, 1), input_shape=(4, 360, 360)),
        resnet,
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    model.load_weights(path)
    return model

In [11]:
model_paths = {
    "mix-gamma-4": "/home/jupyter/wandb/run-20221030_154453-3ua05fk1/files/model-best.h5",
    "h0=0.1": "/home/jupyter/wandb/run-20221030_154818-4lrmhtxo/files/model-best.h5",
    "h0=0.2": "/home/jupyter/wandb/run-20221030_154903-sa3eoy63/files/model-best.h5",
    "mix-gamma-4-more-data": "/home/jupyter/wandb/run-20221101_220800-1sei79yc/files/model-best.h5",
    "mix-gamma-2": "/home/jupyter/wandb/run-20221101_220810-3ao4yzcd/files/model-best.h5",
    "h0=0.1-w1": "/home/jupyter/wandb/run-20221101_220814-37vvoies/files/model-best.h5",
    "h0=0.2-w1": "/home/jupyter/wandb/run-20221101_220818-1gznbinr/files/model-best.h5",
}


In [12]:
# For each model in model_paths, load the model and save it to the models dictionary    
models = {}
for key, path in tqdm(model_paths.items()):
    models[key] = get_model(path)

  0%|          | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/keras/applications/resnet.py:165: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  weights=weights,
2022-11-03 17:37:58.427986: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-03 17:37:58.428049: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-03 17:37:58.428082: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gravitational-waves-kaggle): /proc/driver/nvidia/version does not exist
2022-11-03 17:37:58.428435: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the followin

In [13]:
models

{'mix-gamma-4': <keras.engine.sequential.Sequential at 0x7f7a746d6ed0>,
 'h0=0.1': <keras.engine.sequential.Sequential at 0x7f7a74262990>,
 'h0=0.2': <keras.engine.sequential.Sequential at 0x7f7a7413e8d0>,
 'mix-gamma-4-more-data': <keras.engine.sequential.Sequential at 0x7f7a6c453350>,
 'mix-gamma-2': <keras.engine.sequential.Sequential at 0x7f7a6c043390>,
 'h0=0.1-w1': <keras.engine.sequential.Sequential at 0x7f7a5d2c8990>,
 'h0=0.2-w1': <keras.engine.sequential.Sequential at 0x7f7a7411f650>}

In [14]:
# For each model in models and file path to the file, return the predictions of the model
def get_predictions_for_file(file, models):
    x = read_data(file)
    x = slice_vector(x)
    predictions = {}
    for model_name, model in models.items():
        predictions[model_name] = get_predictions(x, model)
    return predictions

In [15]:
# For each file in pandas dataframe, return the predictions of the model. Name the column with the model name
import gc
def get_predictions_for_df(df, models):
    predictions = {}
    for model_name, model_path in models.items():
        predictions[model_name] = []
    for file in tqdm(df['filename']):
        print(f"Analyzing {file} ... ")
        predictions_for_file = get_predictions_for_file(file, models)
        for model_name, model_path in models.items():
            predictions[model_name].append(predictions_for_file[model_name])
        gc.collect()
    return predictions

In [16]:
def process_predictions(model_name, predictions):
    predictions = predictions[model_name]
    # Get the average of the predictions, std, min, max and median
    
    avg = []
    std = []
    min_ = []
    max_ = []
    median = []
    n = []
    m = []
    for p in predictions:
        avg.append(np.average(p))
        std.append(np.std(p))
        min_.append(np.min(p))
        max_.append(np.max(p))
        median.append(np.median(p))
        # Get the number of p that are greater than 0.5
        n.append(np.sum(p > 0.5))
        # Get the number of p that are less than 0.5
        m.append(np.sum(p < 0.5))
        # Return a dictionary with the above values. Keys have the format "{model_name}_{statistic}"
    return {
        f"{model_name}_avg": avg,
        f"{model_name}_std": std,
        f"{model_name}_min": min_,
        f"{model_name}_max": max_,
        f"{model_name}_median": median,
        f"{model_name}_n": n,
        f"{model_name}_m": m
    }

In [18]:
def process_predictions_for_df(df, models, indx):
    # deep copy df to df_new
    df_new = df.copy(deep=True)
    print(f"Analyzing index {indx} ... ")
    predictions = get_predictions_for_df(df_new, models)
    # return predictions
    for model_name, model_path in models.items():
        df_new = df_new.join(pd.DataFrame(process_predictions(model_name, predictions)))
    df_new.to_csv(f"predictions/predictions-test-{indx}.csv", index=False)
    
    
for i in tqdm(range(0, len(df), 2)):
    process_predictions_for_df(df[i:i+2], models, i)
    gc.collect()


# # Paralelize the above code
# from joblib import Parallel, delayed
# import multiprocessing
# num_cores = multiprocessing.cpu_count()
# results = Parallel(n_jobs=4)(delayed(process_predictions_for_df)(df[i:i+2], models, i) for i in tqdm(range(0, len(df), 2)))




  0%|          | 0/3988 [00:00<?, ?it/s]

Analyzing index 0 ... 


/home/viktorcikojevic/.local/lib/python3.7/site-packages/ipykernel_launcher.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
  app.launch_new_instance()
/home/viktorcikojevic/.local/lib/python3.7/site-packages/ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


Analyzing /input/test/e63b7654c.hdf5 ... 
1/1 [==============================] - 3s 3s/step


Analyzing /input/test/5f92b77cb.hdf5 ... 
1/1 [==============================] - 1s 1s/step


  0%|          | 1/3988 [00:25<28:03:30, 25.34s/it]

Analyzing index 2 ... 


Analyzing /input/test/7fa2c724f.hdf5 ... 


  0%|          | 1/3988 [00:27<30:37:07, 27.65s/it]


KeyboardInterrupt: 